In [ ]:
#Importamos las librerias

In [ ]:
from sklearn.datasets import load_wine #Contiene el dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Guardo el dataset en el objeto wine
wine = load_wine(as_frame=True)

#Convierto el frame de sklearn en un dataframe pandas
dataset = pd.DataFrame(data= np.c_[wine['data'], wine['target']],
                     columns= wine['feature_names'] + ['target'])

dataset

In [ ]:
# Un poco de analisis exploratorio
columnas = dataset.columns
print(columnas)

dataset.describe()


In [ ]:
bar= dataset.groupby(['target'])['target'].count().reset_index(name="n")
bar= pd.DataFrame(bar)

fig = plt.figure(figsize =(4,4))
plt.bar(bar['target'],bar['n'])
plt.xlabel("Target")
plt.ylabel("Cantidad")
plt.title("Distribución de las obs. por target")
plt.show()


#### Observaciones

 - el archivo de datos esta compuesto por variables numéricas lo cuál no implica aplicar una transformación one hot encode a ninguna de las variables.
 - Los predictores son 14 y una variable target.
 - La variable target toma 3 valores, 0, 1 y 2 y la mayoría de las observaciones corresponden a la categoría 1. 

## Preprocesamiento

In [ ]:
#Realizamos escalado y normalizacion de todos los predictores

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

#Selecciono solo los predictores

x = dataset.drop('target', axis=1)

np.random.seed(123)

#Escalado

escala=MinMaxScaler()

escala.fit(x)

x_escalada = escala.transform(x)

#print(x_escalada)

#Normalizado

normalize = Normalizer()

normalize.fit(x)

x_normalizada = normalize.transform(x)

#Estandarizado

estandarizado = StandardScaler()

estandarizado.fit(x)

x_estandarizada = estandarizado.transform(x)


#Corroboramos el estandarizado

print("------------------------------------")
print("Comprobaciones de la estandarizacion")
print("------------------------------------")
print(x_estandarizada.mean(axis=0)) #No es 0, pero es muy cercano a 0
print(x_estandarizada.std(axis=0)) #El desvio es 1 para todos los predictores.



#### Observaciones

 - Con la función StandarScale() hacemos que los predictores se estandaricen, lo que implica que todos pasan a tener media 0 y desviación estándar 1. En el ejercicio la media no es 0, pero es muy cercana y por eso se acepta que esta correcto el estandarizado.
 - Con esta estrategia evitamos que por la unidad de medida del predictor, alguno tenga más peso que el resto.

### Division del dataset en train y test

In [ ]:
# Comenzamos con los datos sin procesar
from sklearn.model_selection import train_test_split


X_train,X_test,y_train, y_test = train_test_split(
dataset.drop('target', axis=1),dataset['target'], random_state=123, test_size= 0.2)


In [ ]:
#Continuamos con los datos preprocesados

from sklearn.model_selection import train_test_split

#Guardamos en el objeto y la variable target

y = dataset['target'].astype(int)

X_train_s,X_test_s,y_train_s, y_test_s = train_test_split(
x_estandarizada,y, random_state=123, test_size= 0.2)



## Entrenamos un modelo Knn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

#Instanciamos el modelo

knn = KNeighborsClassifier(3)

#Entrenamos el modelo

original = knn.fit(X_train,y_train)
preprocesado = knn.fit(X_train_s,y_train_s)

#Hacemos predicciones

y_predicha_original = original.predict(X_test)

y_predicha_preprocesado = preprocesado.predict(X_test_s)

#Obtenemos algunas métricas de error

print("Accurracy")
print("--------------------------------------------")
print("El accuracy obtenido sin escalar es:", round(knn.score(X_test, y_test),3))
print("El accuracy obtenido preprocesado es:", round(knn.score(X_test_s, y_test_s),3))
print("--------------------------------------------")

#Obtenemos el roc_auc

print("ROC_AUC")
print("--------------------------------------------")
print("El roc_auc obtenido sin escalar es:", round(roc_auc_score(y_test,original.predict_proba(X_test),multi_class= 'ovo'),3))
print("El roc_auc obtenido preprocesado es:", round(roc_auc_score(y_test,preprocesado.predict_proba(X_test_s),multi_class= 'ovo'),3))
print("--------------------------------------------")


#### Observaciones

 - Las dos métricas elegidas para evaluar el __modelo KNN__, muestran una __mejor performance__ cuando el modelo utiliza los __datos estandarizados__, llegando a un accuracy de __0.97__ y el roc_auc de __0.99__, vs los 0.22 y 0.5 obtenido con los datos sin estandarizar.

In [ ]:
# Obtenemos la matriz de confusión para ambos dataset (original y preprocesado)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#Matriz de confusion datos originales
cm = confusion_matrix(y_test,y_predicha_original)

#Matriz de confusión datos preprocesados

cm_s = confusion_matrix(y_test,y_predicha_preprocesado)


In [ ]:
fig, ax = plt.subplots(1,2, figsize=(15,10))
ax[0].set_title("Original")
ax[1].set_title("Preprocesado")


disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=knn.classes_)

disp.plot(cmap=plt.cm.GnBu,ax=ax[0])

disp = ConfusionMatrixDisplay(confusion_matrix=cm_s,
                              display_labels=knn.classes_)

disp.plot(cmap=plt.cm.GnBu,ax=ax[1])


#### Observaciones

 - La __matriz de confusión__ muestra de forma más explicita lo que describiamos en el apartado anterior. Cuando el __modelo KNN__ utiliza los datos sin estandarizar (imágen de la izquierda), clasifica las 36 observaciones del dataset de test en 0, cuando __solo 8__ debieron ser clasificadas con este valor. 
 - En cambio, cuando __modelo KNN__ utiliza los datos estandarizados (imágen de la derecha), __clasifica correctamente__ __35 de las 36__ observaciones del __dataset de test__.

## Entrenamos un modelo de DecisionTree

In [ ]:
#importamos el modelo

from sklearn.tree import DecisionTreeClassifier

#generamos una semilla aleatoria

np.random.seed(123)

#Instanciamos el modelo
tree = DecisionTreeClassifier(max_depth=3)

#Entrenamos el modelo

arbol_original = tree.fit(X_train,y_train)
arbol_preprocesado = tree.fit(X_train_s,y_train_s)

#Hacemos predicciones

y_arbol_original = arbol_original.predict(X_test)

y_arbol_preprocesado = arbol_preprocesado.predict(X_test_s)

#Obtenemos algunas métricas de error

print("Accurracy")
print("--------------------------------------------")
print("El accuracy obtenido sin escalar es:", round(tree.score(X_test, y_test),3))
print("El accuracy obtenido preprocesado es:", round(tree.score(X_test_s, y_test_s),3))
print("--------------------------------------------")

#Obtenemos el roc_auc

print("ROC_AUC")
print("--------------------------------------------")
print("El roc_auc obtenido sin escalar es:", round(roc_auc_score(y_test,arbol_original.predict_proba(X_test),multi_class= 'ovo'),3))
print("El roc_auc obtenido preprocesado es:", round(roc_auc_score(y_test_s,arbol_preprocesado.predict_proba(X_test_s),multi_class= 'ovo'),3))
print("--------------------------------------------")


#### Observaciones

 - Al igual que lo observado en el entrenamiento del modelo KNN, aqui podemos observar que tanto el accuracy como el roc_auc muestran mejor performance con los datos preprocesados.

### Optimizamos el hiperparámetro max_depth

In [ ]:
### Optimizamos el parámetro max_depth, utlizando GridSeacrhCV

from sklearn.model_selection import GridSearchCV #grid search

#generamos una semilla aleatoria

np.random.seed(123)

# Hiperoptimizacion del max_depth

param_grid = { 
    'max_depth' : [3,4,5,6,7],
    'criterion' :['gini', 'entropy']
}

#Modelo

arbol = DecisionTreeClassifier()

model_grid = GridSearchCV(estimator=arbol, param_grid=param_grid, cv= 5, refit= True)

#Entrenamos el modelo con datos preprocesados

model_grid_fit = model_grid.fit(X_train_s, y_train_s)



In [ ]:
#Buscamos el mejor estimador

print("Mejor estimador modelo con datos preprocesados")
print("--------------------------------------------")
print(model_grid_fit.best_estimator_)
print("--------------------------------------------")

In [ ]:
#Vemos los errores

resultados = pd.DataFrame(model_grid_fit.cv_results_)
resultados.filter(regex = '(param*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False)



In [ ]:
# Ahora vamos a entrenar nuestro mejor modelo con los parámetros optimizados

#importamos el modelo

from sklearn.tree import DecisionTreeClassifier

#generamos una semilla aleatoria

np.random.seed(123)

#Instanciamos el modelo
tree_opt = DecisionTreeClassifier(max_depth=6,criterion='entropy')

#Entrenamos el modelo

arbol_optimizado = tree_opt.fit(X_train_s,y_train_s)

#Hacemos predicciones

y_arbol_opt = arbol_optimizado.predict(X_test_s)

#Analizamos las metricas de error

print("Accurracy")
print("--------------------------------------------")
print("El accuracy obtenido con el modelo optimizado es:", round(tree_opt.score(X_test_s, y_test_s),3))
print("--------------------------------------------")

print("ROC_AUC")
print("--------------------------------------------")
print("El roc_auc obtenido con el modelo optimizado es:", round(roc_auc_score(y_test_s,arbol_optimizado.predict_proba(X_test_s),multi_class= 'ovo'),3))
print("--------------------------------------------")



#### Observaciones

 - Se __optimizaron 2 hiperparámetros__ del modelo de DecisionTree, el __max_depth__ y el __criterio de división__ mediante la estrategia __GridSearchCV__.
 - La tabla _resultados_ muestra los modelos entrenados con distintas combicaciones de los hiperparámetros. Asmismo muestra el promedio del accuracy ordenados de mayor a menor.
 - De allí podemos saber que el modelo más optimo es el que setea los parámtros __max_depth__ en __6__ y __criterion__ __entropy__ .
 - En este caso particuar la __optimización de los parámetros__ no implicó una mejora en las métricas de error, incluso se obtuvo un menor accuracy.

### Vemos la importancia de los predictores

In [ ]:
#Vemos la importancia de los predictores

importancia_predictores = pd.DataFrame(
                            {'predictor': x.columns,
                             'importancia':tree.feature_importances_}
                            )
importancia_predictores.sort_values('importancia', ascending=False)

#### Observaciones

 - Aquí podemos observar que los __feature__ _proline_ y _od280/od315_of_diluted_wines_ son los que mayor importancia tuvieron en el modelo.

## Aplicamos un modelo de reducción de dimensionalidad

### Utilizamos el Análisis de la componente principal

In [ ]:
from sklearn.decomposition import PCA
import mglearn

#Instanciamos el modelo PC
pca=PCA(n_components=2)

#Entrenamoe el modelo
pca.fit(x_estandarizada)

#Transformamos los datos estandarizados
transformados = pca.transform(x_estandarizada)
print(x.shape)
print(transformados.shape)

#hacemos un gráfico


mglearn.discrete_scatter(transformados[:,0], transformados[:,1], dataset['target'])
plt.legend(dataset['target'].unique(), loc='lower right')
plt.xlabel('Componente pca 1')
plt.ylabel('Componente pca 2')

#Creamos un dataframe con los predictores que quedan incluidos en ambos componentes principales

predictores = pd.DataFrame(pca.components_.T, columns= ['PC-1','PC-2'], index= x.columns)

predictores.sort_values('PC-1',ascending = False)

In [ ]:
print('----------------------------------------------------')
print('Porcentaje de varianza explicada por cada componente')
print('----------------------------------------------------')
print(pca.explained_variance_ratio_)
print('--------------------------------------------------------------')
print('Porcentaje de varianza acumulada explicada por cada componente')
print('--------------------------------------------------------------')
print(pca.explained_variance_ratio_.cumsum())

#### Observaciones

 - En el cuadro anterior podemos observar cuáles __feature__ dentro de los componentes explican la mayor cantidad de varianza.
 - En nuestro análisis podemos ver que las variables _flavonoid_, _total_phenols_ y _od280/od315_of_diluted_wines_ tienen el mayor peso en la PC-1. Mientras que en PC-2 tienen más peso _color_intensity_, _alcohol_ y _proline_. 

## Aplicamos el modelo KMeans

In [ ]:
#Importamos la libreria
import time
from sklearn.cluster import KMeans
from sklearn import metrics

#plantamos una semilla aleatoria

np.random.seed(123)

#Instanciamos el modelo

model_k_means = KMeans(n_clusters=3, max_iter=2000) 

#Instanciamos el momento 0
t0 = time.time()

#Entrenamos el modelo
model_k_means.fit(transformados)

#Capturamos el tiempo luego de entrenar el modelo menos el momento 0
t_batch = time.time() - t0


print("-------------------------")
print("---Tiempo de ejecución---")
print("-------",round(t_batch,2),"--------")
print("-------------------------")

#Generamos las predicciones

pred_k_means = model_k_means.predict(transformados)

#Obtenemos las métricas de error

score=metrics.adjusted_rand_score(dataset['target'], pred_k_means)

print("-----------------------------")
print("------Indice Rand------------")
print("El score obtenido es: ",round(score,3))
print("-----------------------------")

#Graficamos las predicciones


plt.scatter(transformados[:, 0], transformados[:, 1], c=pred_k_means)

plt.show()

## Aplicamos el modelo KMeans mini-batches 

In [ ]:
# importamos el modelo de la libreria cluster

from sklearn.cluster import MiniBatchKMeans
from sklearn import metrics
import time

#plantamos una semilla aleatoria

np.random.seed(123)


#Instanciamos el momento 0
t1 = time.time()

#Instanciamos el mismo modelo pero Mini batches

k_means_mb = MiniBatchKMeans(n_clusters=3, max_iter=2000)

#Capturamos el tiempo luego de entrenar el modelo menos el momento 0
t_batch_1 = time.time() - t1

print("-------------------------")
print("---Tiempo de ejecución---")
print("---------",round(t_batch_1,2),"----------")
print("-------------------------")

#Entrenamos el modelo tulizando los datos escalados

k_means_mb.fit(transformados)

#Creamos las predicciones del modelo

pred_k_means_mb = k_means_mb.predict(transformados)

#Obtenemos las métricas de error

score_mb= metrics.adjusted_rand_score(dataset['target'], pred_k_means_mb)

print("-----------------------------")
print("------Indice Rand------------")
print("El score obtenido es: ",round(score_mb,3))
print("-----------------------------")

#Graficamos las predicciones


plt.scatter(transformados[:, 0], transformados[:, 1], c=pred_k_means_mb)

plt.show()



#### Observaciones

 - Podemos observar que el modelo K-Means Mini Batch nos permite agrupar los datos en cluster con una menor precisión que K-means (score 0.847 vs 0.895). Ahora bien, desde el punto de vista del tiempo de ejecución obtenido con la __libreria time__, podemos observar que el modelo K-means tardo 0.4 s y el modelo K-means Mini Batch tarda 0.0 s. 
 - En conclusión, el modelo __K-means Mini Batch__ es más __eficiente__ obteniendo métricas de error solo __levemente mayores__ al __modelo K-means__.

### Bonus Track

Quizás un primer paso que debería haber implementado es utilizar una matriz de correlación para observar la existencia de __colinealidad__ y consecuencia excluir los predictores que este correlacionados.
En esta etapa analizaremos los predictores mediante una __matriz de correlación__.

In [ ]:
#Armamos un data frame con los predictores
data_corr = pd.DataFrame(x_estandarizada, columns = x.columns)

#Mostramos la matriz de correlacion

corr_mat = data_corr.corr()

#Graficamos la matriz de correlacion

import seaborn as sn

plt.figure(figsize=(10, 8))


sn.heatmap(corr_mat, annot=True,cmap="crest")

plt.show()

print(x.columns)

#### Observaciones

 - En la matriz se puede observar una fuerte correlación entre las siguientes variables independientes/predictores:
     - proline y alcohol: 0.64
     - od280/od315_of_diluted_wines y total_phenols
     - proanthocyanins y total_phenols
     - flavanoids y total_phenols
     - od280/od315_of_diluted_wines y flavanoids
     
 - Teniendo en cuenta lo anterior podría excluirse del modelo __total_phenols__, __flavanoids__, __od280/od315_of_diluted_wines__. También, como se hizo en este TP, aplicar un __PCA__ ayuda a reducir la dimensionalidad y por lo tanto controlar lo __colinealidad/multicolinealidad__.